In [1]:
import sys
import pandas as pd
from load_data import get_data, save_pkl, load_pkl
from processing_helpers import get_control_segments, get_power_spec, get_peak_fits
from tally_helper import TallyWindow
# from plot_window import plot_processing
import pickle as pkl
import numpy as np
import DataProcessingTools as DPT
import os
import matplotlib.pyplot as plt
from fooof import FOOOF
from ipywidgets import widgets, HBox, VBox, Output
import mne
from scipy.stats import sem, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

%matplotlib qt

In [2]:
# Global variables to store the session data
lfp_df = pd.DataFrame()
ch_num_list = []
lfp_mne = []
session_start_time = 0
markers = []
timeStamps = []
sampling_frequency = 0
segments_df = pd.DataFrame()


In [3]:
def load_data(day):
    """
    Load LFP data for a given day.
    """
    global lfp_df, ch_num_list, lfp_mne, session_start_time, markers, timeStamps, sampling_frequency
    print(f"Loading data for {day}...")
    lfp_df, ch_num_list, lfp_mne, session_start_time, markers, timeStamps, sampling_frequency = get_data(day)

    if lfp_df is not None:
        print(f"Loaded {len(ch_num_list)} channels.")
        print(lfp_df.head())
    else:
        print("Failed to load data.")

def get_segment(segments_df, window_size, method='welch', max_n_peaks=3, peak_threshold=2, peak_fit_range=[1,12]):
    """
    Extract segments from the LFP data using the specified window size.
    """
    if lfp_df.empty or not markers or not timeStamps:
        print("Please load data before extracting segments.")
        return

    segments_df = get_control_segments(lfp_df, markers, timeStamps, window_size=window_size)
    print(f"Extracted {len(segments_df)} segments.")

    frequencies = []
    psd_list = []
    p_stds_l = []
    p_means_l = []
    peak_gauss_l = []
    flat_spec_l = []
    flat_spec_og_l = []
    ap_fit_og_l = []
    peaks_l, peak_freqs_l = [], []

    count = 0
    segments_list = segments_df['segment'].values  # Convert df to array for faster processing

    for segment in segments_list:
        freq, psd = get_power_spec(segment, sampling_frequency, method=method)
        frequencies.append(freq)


        psd, ap_fit_og, flat_spec_og, p_stds, p_means, peak_gauss, flat_specs, peaks, peak_freqs= get_peak_fits(psd, freq, peak_fit_range, max_n_peaks=max_n_peaks, peak_threshold=peak_threshold)
        psd_list.append(psd)
        p_stds_l.append(p_stds)
        p_means_l.append(p_means)
        peak_gauss_l.append(peak_gauss)
        flat_spec_l.append(flat_specs)
        flat_spec_og_l.append(flat_spec_og)
        ap_fit_og_l.append(ap_fit_og)
        peaks_l.append(peaks)
        peak_freqs_l.append(peak_freqs)

        count += 1
        print(f"Processed segment {count}")

    segments_df['psd'] = psd_list
    segments_df['freq'] = frequencies
    segments_df['peak_stds'] = p_stds_l
    segments_df['peak_means'] = p_means_l
    segments_df['peak_gauss'] = peak_gauss_l
    segments_df['flat_specs'] = flat_spec_l
    segments_df['flat_spec_og'] = flat_spec_og_l
    segments_df['ap_fit_og'] = ap_fit_og_l
    segments_df['peaks'] = peaks_l
    segments_df['peak_freqs'] = peak_freqs_l
    print("Segments processed successfully.")
    return segments_df

def show_tally():
    """
    Display tally information for the segments DataFrame.
    """
    if segments_df.empty:
        print("Please generate segments before viewing the tally.")
        return

    # Example of tally logic; customize as needed
    print("Tally of segments by channel:")
    print(segments_df.groupby('channel').size())
    TallyWindow(segments_df, ch_num_list).show()


In [15]:
# Load Raw Data
day = "20181101"
load_data(day)

Loading data for 20181101...
Creating RawArray with float64 data, n_channels=26, n_times=5114500
    Range : 0 ... 5114499 =      0.000 ...  5114.499 secs
Ready.
Loaded 26 channels.
  channel                                           lfp_data
0     009  [32.99068, 61.675568, 80.906586, 87.342415, 84...
1     019  [-11.008957, -2.4303188, 2.42602, 2.5361316, -...
2     021  [23.140007, 30.881031, 34.674755, 33.311707, 2...
3     022  [18.339674, 43.174232, 58.28896, 59.78019, 51....
4     023  [21.936518, 37.05075, 45.68002, 45.50032, 39.2...


In [16]:
# Extract segments with a specific window size and process
window_size = 1000  # e.g., 1000 ms
peak_fit_range = [1,14]
segments_df = get_segment(lfp_df, window_size, method='welch', max_n_peaks=3, peak_threshold=1.76, peak_fit_range=peak_fit_range)

Extracted 10374 segments.
Processed segment 1
Processed segment 2
Processed segment 3
Processed segment 4
Processed segment 5
Processed segment 6
Processed segment 7
Processed segment 8
Processed segment 9
Processed segment 10
Processed segment 11
Processed segment 12
Processed segment 13
Processed segment 14
Processed segment 15
Processed segment 16
Processed segment 17
Processed segment 18
Processed segment 19
Processed segment 20
Processed segment 21
Processed segment 22
Processed segment 23
Processed segment 24
Processed segment 25
Processed segment 26
Processed segment 27
Processed segment 28
Processed segment 29
Processed segment 30
Processed segment 31
Processed segment 32
Processed segment 33
Processed segment 34
Processed segment 35
Processed segment 36
Processed segment 37
Processed segment 38
Processed segment 39
Processed segment 40
Processed segment 41
Processed segment 42
Processed segment 43
Processed segment 44
Processed segment 45
Processed segment 46
Processed segment

In [17]:
save_pkl(f"/Users/liuyuanwei/Desktop/Data Processed/{day}/{day}_control.pkl",segments_df)

Data successfully exported to /Users/liuyuanwei/Desktop/Data Processed/20181101/20181101_control.pkl
